## Import Library

In [33]:
# Import basic library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import re

# Import library for load env & connect database
import supabase
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2 import sql

## Import Data

## Develop SQL Logic

In [34]:
conn = psycopg2.connect(
    host=os.getenv("host"),
    port=os.getenv("port"),
    dbname=os.getenv("database"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    sslmode="require",
    connect_timeout=30,
)

### CTE to Get Employee Benchmark Data

In [35]:
# Load environment variables from a .env file if present
load_dotenv()  # Ensures os.getenv(...) can pick up values from local .env

# Create function to get employees benchmark data
def employees_benchmark_data(employee_ids, schema: str = "public") -> pd.DataFrame:
    """
    Build a consolidated employee benchmark dataset by joining:
      - Employees master (IDs and dimension FKs)
      - Latest competencies score per employee (yearly history, take the newest year)
      - Psychological profiles (pauli, DISC word, MBTI, IQ, GTQ, TIKI)
      - Strengths (top-ranked theme per employee)
      - PAPI scores (scale and score)
      - Dimension tables to resolve human-readable names:
          * directorate name    (dim_directorates.name)
          * role/position name  (dim_positions.name)
          * grade name          (dim_grades.name)

    Output columns:
      employee_id, full_name, directorate, role, grade, score, pauli,
      disc_1, disc_2, mbti_1, mbti_2, mbti_3, mbti_4,
      iq, gtq, tiki, strengths, scale_code, papi_scores
    """
    # If no IDs are provided, return an empty DataFrame with the expected schema
    if not employee_ids:
        return pd.DataFrame(columns=[
            "employee_id","full_name","directorate","role","grade","score",
            "pauli","disc_1","disc_2","mbti_1","mbti_2","mbti_3","mbti_4",
            "iq","gtq","tiki","strengths","scale_code","papi_scores"
        ])

    # Sanity check for schema name: allow alphanumerics and underscores only
    assert schema.replace("_", "").isalnum(), "Invalid schema name"

    # SQL query
    qry = f"""
    WITH                                                          -- Begin CTE section
    ids AS (                                                       -- CTE: expand provided IDs
      SELECT UNNEST(%s::text[]) AS employee_id                    -- Turn the text[] parameter into one row per employee_id
    ),                                                            -- Close CTE; continue to next
    employees_base AS (                                           -- CTE: base employees limited to requested IDs
      SELECT                                                      -- Select columns needed downstream
        e.employee_id,                                            -- Employee primary key
        e.fullname AS full_name,                                  -- Human-readable full name
        e.nip,                                                    -- Internal employee number
        e.directorate_id,                                         -- FK to directorates dimension
        e.position_id,                                            -- FK to positions (role) dimension
        e.grade_id                                                -- FK to grades dimension
      FROM {schema}.employees e                                   -- Source table: employees master
      JOIN ids i ON i.employee_id = e.employee_id::text           -- Keep only employees present in the input ID list
    ),                                                            -- Close CTE; continue
    competencies_latest AS (                                      -- CTE: latest competency score per employee (by year)
      SELECT DISTINCT ON (cy.employee_id)                         -- Keep only the newest row per employee_id
            cy.employee_id,                                      -- Employee key
            cy.score,                                            -- Competency score
            cy.year                                              -- Year of the score
      FROM {schema}.competencies_yearly cy                        -- Source table: yearly competencies
      JOIN ids i ON i.employee_id = cy.employee_id::text          -- Restrict to requested employees
      WHERE cy.year IS NOT NULL                                   -- Ignore rows that don't have a valid year
      ORDER BY cy.employee_id, cy.year DESC                       -- Sort so the newest year comes first per employee
    ),                                                            -- Close CTE; continue
    profiles_psych_f AS (                                         -- CTE: psychological profiles (if any)
      SELECT                                                      -- Select psychometric fields
        p.employee_id,                                            -- Employee key
        p.pauli,                                                  -- Pauli score
        p.disc_word,                                              -- DISC word (e.g., "D-I")
        p.mbti,                                                   -- MBTI type (e.g., "INTJ")
        p.iq,                                                     -- IQ score
        p.gtq,                                                    -- GTQ score
        p.tiki                                                    -- TIKI score
      FROM {schema}.profiles_psych p                              -- Source table: profiles_psych
      JOIN ids i ON i.employee_id = p.employee_id::text           -- Restrict to requested employees
    ),                                                            -- Close CTE; continue
    strengths_rank AS (                                           -- CTE: rank strengths to find the top theme
      SELECT                                                      -- Compute row_number over strengths per employee
        s.employee_id,                                            -- Employee key
        s.rank,                                                   -- Strength rank value
        s.theme,                                                  -- Strength theme label
        ROW_NUMBER() OVER (PARTITION BY s.employee_id             -- Window: restart per employee
                          ORDER BY s.rank ASC, s.theme) AS rn     -- rn=1 marks the top-ranked (ties broken by theme)
      FROM {schema}.strengths s                                   -- Source table: strengths
      JOIN ids i ON i.employee_id = s.employee_id::text           -- Restrict to requested employees
    ),                                                            -- Close CTE; continue
    strengths_top AS (                                            -- CTE: keep only the top strength per employee
      SELECT employee_id, theme                                   -- Output: employee and their top theme
      FROM strengths_rank                                         -- Source: ranked strengths
      WHERE rn = 1                                                -- Filter to top-ranked row
    ),                                                            -- Close CTE; continue
    papi_f AS (                                                   -- CTE: PAPI scores per scale (may be multiple rows)
      SELECT                                                      -- Select PAPI details
        ps.employee_id,                                           -- Employee key
        ps.scale_code,                                            -- PAPI scale identifier
        ps.score AS papi_scores                                   -- PAPI score value
      FROM {schema}.papi_scores ps                                -- Source table: papi_scores
      JOIN ids i ON i.employee_id = ps.employee_id::text          -- Restrict to requested employees
    ),                                                            -- Close CTE; continue
    -- Dimension CTEs: map IDs to display names
    dim_directorates_f AS (                                       -- CTE: directorate names
      SELECT directorate_id, name AS directorate                  -- Map directorate_id -> directorate name
      FROM {schema}.dim_directorates                              -- Source dimension: directorates
    ),                                                            -- Close CTE; continue
    dim_positions_f AS (                                          -- CTE: position/role names
      SELECT position_id, name AS role                            -- Map position_id -> role name
      FROM {schema}.dim_positions                                 -- Source dimension: positions
    ),                                                            -- Close CTE; continue
    dim_grades_f AS (                                             -- CTE: grade names
      SELECT grade_id, name AS grade                              -- Map grade_id -> grade name
      FROM {schema}.dim_grades                                    -- Source dimension: grades
    ),                                                            -- Close CTE; continue
    emp_with_profiles AS (                                        -- CTE: employees enriched with psych profiles
      SELECT                                                      -- Select base employee fields + profile columns
        eb.employee_id,                                           -- Employee key
        eb.full_name,                                             -- Name
        eb.nip,                                                   -- Internal number
        eb.directorate_id,                                        -- FK directorate
        eb.position_id,                                           -- FK position
        eb.grade_id,                                              -- FK grade
        pp.pauli,                                                 -- Pauli (nullable)
        pp.disc_word,                                             -- DISC (nullable)
        pp.mbti,                                                  -- MBTI (nullable)
        pp.iq,                                                    -- IQ (nullable)
        pp.gtq,                                                   -- GTQ (nullable)
        pp.tiki                                                   -- TIKI (nullable)
      FROM employees_base eb                                      -- Source: filtered employees
      LEFT JOIN profiles_psych_f pp USING (employee_id)           -- Optional join: keep employees without profiles
    ),                                                            -- Close CTE; continue
    emp_with_strengths AS (                                       -- CTE: add top strength theme
      SELECT                                                      -- Carry all previous fields and add theme
        ewp.*,                                                    -- All columns from emp_with_profiles
        st.theme AS strengths                                     -- Top strength theme (nullable)
      FROM emp_with_profiles ewp                                  -- Source: previous CTE
      LEFT JOIN strengths_top st USING (employee_id)              -- Optional join: attach top theme if any
    ),                                                            -- Close CTE; continue
    emp_with_papi AS (                                            -- CTE: add PAPI scale/score (may add row multiplicity)
      SELECT                                                      -- Carry all previous fields and add PAPI
        ews.*,                                                    -- All columns from emp_with_strengths
        pf.scale_code,                                            -- PAPI scale code (nullable)
        pf.papi_scores                                            -- PAPI score (nullable)
      FROM emp_with_strengths ews                                 -- Source: previous CTE
      LEFT JOIN papi_f pf USING (employee_id)                     -- Optional join: attach PAPI per scale
    ),                                                            -- Close CTE; continue
    -- Join human-readable names from dimensions
    emp_with_names AS (                                           -- CTE: attach directorate/role/grade names
      SELECT                                                      -- Carry all previous fields + names
        ewp.*,                                                    -- All columns so far
        dd.directorate,                                           -- Directorate display name
        dp.role,                                                  -- Role/position display name
        dg.grade                                                  -- Grade display name
      FROM emp_with_papi ewp                                      -- Source: previous CTE
      LEFT JOIN dim_directorates_f dd ON dd.directorate_id = ewp.directorate_id  -- Join directorate name
      LEFT JOIN dim_positions_f    dp ON dp.position_id    = ewp.position_id     -- Join role name
      LEFT JOIN dim_grades_f       dg ON dg.grade_id       = ewp.grade_id         -- Join grade name
    ),                                                            -- Close CTE; continue
    -- Parse MBTI/DISC into separate columns
    parsed_splits AS (                                            -- CTE: per-employee DISC/MBTI splits
      SELECT                                                      -- Compute derived columns
        ewp.employee_id,                                          -- Employee key
        NULLIF(split_part(ewp.disc_word, '-', 1), '') AS disc_1,  -- DISC first component; empty string -> NULL
        NULLIF(split_part(ewp.disc_word, '-', 2), '') AS disc_2,  -- DISC second component; empty string -> NULL
        CASE                                                      -- MBTI letter 1 -> Introversion/Extraversion
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL  -- Guard for invalid MBTI
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),1,1)) = 'I' THEN 'Introversion' -- I => Introversion
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),1,1)) = 'E' THEN 'Extraversion' -- E => Extraversion
          ELSE NULL                                               -- Anything else -> NULL
        END AS mbti_1,                                            -- Aliased as mbti_1
        CASE                                                      -- MBTI letter 2 -> Sensing/Intuition
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL  -- Guard for invalid MBTI
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),2,1)) = 'S' THEN 'Sensing'  -- S => Sensing
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),2,1)) = 'N' THEN 'Intuition'-- N => Intuition
          ELSE NULL                                               -- Anything else -> NULL
        END AS mbti_2,                                            -- Aliased as mbti_2
        CASE                                                      -- MBTI letter 3 -> Thinking/Feeling
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL  -- Guard for invalid MBTI
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),3,1)) = 'T' THEN 'Thinking' -- T => Thinking
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),3,1)) = 'F' THEN 'Feeling'  -- F => Feeling
          ELSE NULL                                               -- Anything else -> NULL
        END AS mbti_3,                                            -- Aliased as mbti_3
        CASE                                                      -- MBTI letter 4 -> Judging/Perceiving
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL  -- Guard for invalid MBTI
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),4,1)) = 'J' THEN 'Judging'  -- J => Judging
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),4,1)) = 'P' THEN 'Perceiving' -- P => Perceiving
          ELSE NULL                                               -- Anything else -> NULL
        END AS mbti_4                                             -- Aliased as mbti_4
      FROM emp_with_names ewp                                     -- Source rows to parse
    ),                                                            -- Close CTE; continue
    -- Assemble raw output (may have duplicates)
    final_raw AS (                                                -- CTE: compose raw result rows
      SELECT                                                      -- Select normalized outputs
        ewp.employee_id,                                          -- Employee key
        ewp.full_name,                                            -- Name
        ewp.directorate,                                          -- Directorate display name
        ewp.role,                                                 -- Role/position display name
        ewp.grade,                                                -- Grade display name
        COALESCE(cl.score, 0)  AS score,                          -- Latest competency score; default 0
        COALESCE(ewp.pauli, 0) AS pauli,                          -- Pauli score; default 0
        ps.disc_1,                                                -- DISC part 1
        ps.disc_2,                                                -- DISC part 2
        ps.mbti_1,                                                -- MBTI dim 1
        ps.mbti_2,                                                -- MBTI dim 2
        ps.mbti_3,                                                -- MBTI dim 3
        ps.mbti_4,                                                -- MBTI dim 4
        COALESCE(ewp.iq,  0)   AS iq,                             -- IQ; default 0
        COALESCE(ewp.gtq, 0)   AS gtq,                            -- GTQ; default 0
        COALESCE(ewp.tiki,0)   AS tiki,                           -- TIKI; default 0
        ewp.strengths          AS strengths,                      -- Top strength theme
        ewp.scale_code,                                           -- PAPI scale code
        COALESCE(ewp.papi_scores, 0) AS papi_scores               -- PAPI score; default 0
      FROM emp_with_names ewp                                     -- Source: names and PAPI attached
      LEFT JOIN competencies_latest cl USING (employee_id)        -- Add the latest competency score
      LEFT JOIN parsed_splits     ps USING (employee_id)          -- Add DISC/MBTI split columns
    )                                                             -- Close CTE; continue
    -- Final select with de-duplication
    SELECT DISTINCT                                               -- Remove duplicate rows in the projection
      employee_id,                                                -- Employee key
      full_name,                                                  -- Name
      directorate,                                                -- Directorate
      role,                                                       -- Role/position
      grade,                                                      -- Grade
      score,                                                      -- Latest competency score
      pauli,                                                      -- Pauli score
      disc_1,                                                     -- DISC part 1
      disc_2,                                                     -- DISC part 2
      mbti_1,                                                     -- MBTI dim 1
      mbti_2,                                                     -- MBTI dim 2
      mbti_3,                                                     -- MBTI dim 3
      mbti_4,                                                     -- MBTI dim 4
      iq,                                                         -- IQ
      gtq,                                                        -- GTQ
      tiki,                                                       -- TIKI
      strengths,                                                  -- Top strength theme
      scale_code,                                                 -- PAPI scale code
      papi_scores                                                 -- PAPI score
    FROM final_raw                                                -- Read from the assembled raw CTE
    ORDER BY employee_id, full_name;                              -- Stable, readable ordering by employee id & name
    """

    # Open a PostgreSQL connection using env vars (works well for Supabase)
    conn = psycopg2.connect(
        host=os.getenv("host"),         # DB hostname
        port=os.getenv("port"),         # DB port (e.g., 5432)
        dbname=os.getenv("database"),   # Database name
        user=os.getenv("user"),         # Username
        password=os.getenv("password"), # Password/secret
        sslmode="require",              # Enforce SSL
        connect_timeout=30,             # Bound connection wait time
    )
    try:
        # Use context manager for the cursor to ensure proper cleanup
        with conn.cursor() as cur:
            # Execute the parameterized SQL; pass employee_ids as a single argument
            cur.execute(qry, (employee_ids,))
            # Retrieve all rows from the result set
            rows = cur.fetchall()
            # Extract column names from the cursor metadata
            cols = [d[0] for d in cur.description]
        # Build a DataFrame with the rows and column names
        return pd.DataFrame(rows, columns=cols)
    finally:
        # Always close the DB connection
        conn.close()

In [36]:
# List example of benchmark employees_id
employee_ids = ['EMP100011', 'DUP3015']

# Call function employees_benchmark_data
df_employee_benchmarks = employees_benchmark_data(employee_ids, schema="public")
df_employee_benchmarks

,employee_id,full_name,directorate,role,grade,score,pauli,disc_1,disc_2,mbti_1,mbti_2,mbti_3,mbti_4,iq,gtq,tiki,strengths,scale_code,papi_scores
0,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_A,5.0
1,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_B,4.0
2,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_C,3.0
3,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_D,4.0
4,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_E,9.0
5,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_F,2.0
6,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_G,8.0
7,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_I,9.0
8,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_K,1.0
9,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,3.0,81,Conscientious,Dominant,Extraversion,Sensing,Feeling,Perceiving,87.0,22.0,6,Responsibility,Papi_L,7.0


### CTE to Get All Employee Data

In [37]:
# Create function to get all employee data
def get_all_employee_data(schema: str = "public") -> pd.DataFrame:
    """
    Retrieve ALL employees with:
      - latest competencies score per employee,
      - psychological profiles (pauli, DISC word, MBTI, IQ, GTQ, TIKI),
      - top-ranked strength,
      - PAPI scale & score,
      then join to dimension tables for human-readable names (directorate, role, grade).

    Output columns:
      employee_id, full_name, directorate, role, grade, score, pauli,
      disc_1, disc_2, mbti_1, mbti_2, mbti_3, mbti_4,
      iq, gtq, tiki, strengths, scale_code, papi_scores
    """
    assert schema.replace("_", "").isalnum(), "Invalid schema name"

    # SQL Query
    qry = f"""
    WITH                                                         -- Start CTE block
    employees_base AS (                                          -- CTE: base employees (all rows)
      SELECT                                                     -- Select base fields
        e.employee_id,                                           -- Employee primary key
        e.fullname AS full_name,                                 -- Human-readable full name
        e.nip,                                                   -- Internal employee number
        e.directorate_id,                                        -- FK to directorates
        e.position_id,                                           -- FK to positions (role)
        e.grade_id                                               -- FK to grades
      FROM {schema}.employees e                                  -- Source: employees master (entire table)
    ),                                                           -- End CTE employees_base
    competencies_latest AS (                                     -- CTE: pick latest competency per employee
      SELECT DISTINCT ON (cy.employee_id)                        -- Keep one row per employee_id (latest by year)
            cy.employee_id,                                     -- Employee key
            cy.score,                                           -- Competency score
            cy.year                                             -- Year of the score
      FROM {schema}.competencies_yearly cy                       -- Source: competencies by year
      WHERE cy.year IS NOT NULL                                  -- Exclude rows with NULL year
      ORDER BY cy.employee_id, cy.year DESC                      -- Sort so newest year is chosen by DISTINCT ON
    ),                                                           -- End CTE competencies_latest
    profiles_psych_f AS (                                        -- CTE: psychological profiles
      SELECT                                                     -- Select psych fields
        p.employee_id,                                           -- Employee key
        p.pauli,                                                 -- Pauli score
        p.disc_word,                                             -- DISC word (e.g., "D-I")
        p.mbti,                                                  -- MBTI type (e.g., "INTJ")
        p.iq,                                                    -- IQ score
        p.gtq,                                                   -- GTQ score
        p.tiki                                                   -- TIKI score
      FROM {schema}.profiles_psych p                             -- Source: psych profiles table
    ),                                                           -- End CTE profiles_psych_f
    strengths_rank AS (                                          -- CTE: rank strengths per employee
      SELECT                                                     -- Compute row numbers by employee
        s.employee_id,                                           -- Employee key
        s.rank,                                                  -- Rank of the strength
        s.theme,                                                 -- Strength theme
        ROW_NUMBER() OVER (PARTITION BY s.employee_id            -- Row number restart per employee
                          ORDER BY s.rank ASC, s.theme) AS rn    -- rn=1 marks top theme (tie-break by theme)
      FROM {schema}.strengths s                                  -- Source: strengths table
    ),                                                           -- End CTE strengths_rank
    strengths_top AS (                                           -- CTE: keep only top strength per employee
      SELECT employee_id, theme                                  -- Output key + top theme
      FROM strengths_rank                                        -- Source: ranked strengths
      WHERE rn = 1                                               -- Keep top-ranked only
    ),                                                           -- End CTE strengths_top
    papi_f AS (                                                  -- CTE: PAPI scores (may be multiple per employee)
      SELECT                                                     -- Select PAPI fields
        ps.employee_id,                                          -- Employee key
        ps.scale_code,                                           -- PAPI scale code
        ps.score AS papi_scores                                  -- PAPI score value
      FROM {schema}.papi_scores ps                               -- Source: PAPI scores table
    ),                                                           -- End CTE papi_f
    -- Dimension CTEs: map IDs to display names                  -- Note: next CTEs map IDs -> names
    dim_directorates_f AS (                                      -- CTE: directorate names
      SELECT directorate_id, name AS directorate                 -- Map directorate_id to name
      FROM {schema}.dim_directorates                             -- Source: directorates dimension
    ),                                                           -- End CTE dim_directorates_f
    dim_positions_f AS (                                         -- CTE: role/position names
      SELECT position_id, name AS role                           -- Map position_id to role name
      FROM {schema}.dim_positions                                -- Source: positions dimension
    ),                                                           -- End CTE dim_positions_f
    dim_grades_f AS (                                            -- CTE: grade names
      SELECT grade_id, name AS grade                             -- Map grade_id to grade name
      FROM {schema}.dim_grades                                   -- Source: grades dimension
    ),                                                           -- End CTE dim_grades_f
    emp_with_profiles AS (                                       -- CTE: join base with psych profiles
      SELECT                                                     -- Select base + psych fields
        eb.employee_id,                                          -- Employee key
        eb.full_name,                                            -- Full name
        eb.nip,                                                  -- Employee number
        eb.directorate_id,                                       -- FK directorate
        eb.position_id,                                          -- FK position
        eb.grade_id,                                             -- FK grade
        pp.pauli,                                                -- Pauli (nullable)
        pp.disc_word,                                            -- DISC word (nullable)
        pp.mbti,                                                 -- MBTI (nullable)
        pp.iq,                                                   -- IQ (nullable)
        pp.gtq,                                                  -- GTQ (nullable)
        pp.tiki                                                  -- TIKI (nullable)
      FROM employees_base eb                                     -- Source: all employees
      LEFT JOIN profiles_psych_f pp USING (employee_id)          -- Left join: keep employees without profiles
    ),                                                           -- End CTE emp_with_profiles
    emp_with_strengths AS (                                      -- CTE: add top strength theme
      SELECT                                                     -- Carry all cols + theme
        ewp.*,                                                   -- All columns from previous CTE
        st.theme AS strengths                                    -- Top strength theme (nullable)
      FROM emp_with_profiles ewp                                 -- Source: employees with profiles
      LEFT JOIN strengths_top st USING (employee_id)             -- Left join: attach top theme
    ),                                                           -- End CTE emp_with_strengths
    emp_with_papi AS (                                           -- CTE: add PAPI scale and score
      SELECT                                                     -- Carry all cols + PAPI fields
        ews.*,                                                   -- All columns so far
        pf.scale_code,                                           -- PAPI scale code (nullable)
        pf.papi_scores                                           -- PAPI score (nullable)
      FROM emp_with_strengths ews                                -- Source: employees with strengths
      LEFT JOIN papi_f pf USING (employee_id)                    -- Left join: attach PAPI rows
    ),                                                           -- End CTE emp_with_papi
    -- Join human-readable names from dimensions                 -- Note: join human-readable names
    emp_with_names AS (                                          -- CTE: attach names from dimensions
      SELECT                                                     -- Carry all cols + names
        ewp.*,                                                   -- All columns so far
        dd.directorate,                                          -- Directorate name
        dp.role,                                                 -- Role/position name
        dg.grade                                                 -- Grade name
      FROM emp_with_papi ewp                                     -- Source: employees with PAPI
      LEFT JOIN dim_directorates_f dd ON dd.directorate_id = ewp.directorate_id  -- Join directorate name
      LEFT JOIN dim_positions_f    dp ON dp.position_id    = ewp.position_id     -- Join role name
      LEFT JOIN dim_grades_f       dg ON dg.grade_id       = ewp.grade_id         -- Join grade name
    ),                                                           -- End CTE emp_with_names
    -- Parse MBTI/DISC into separate columns                     -- Note: normalize MBTI & DISC
    parsed_splits AS (                                           -- CTE: derive DISC/MBTI columns
      SELECT                                                     -- Compute parsed fields
        ewp.employee_id,                                         -- Employee key
        NULLIF(split_part(ewp.disc_word, '-', 1), '') AS disc_1, -- DISC first component ('' -> NULL)
        NULLIF(split_part(ewp.disc_word, '-', 2), '') AS disc_2, -- DISC second component ('' -> NULL)
        CASE                                                     -- MBTI letter 1 -> Intro/Extra
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL -- Guard for invalid MBTI
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),1,1)) = 'I' THEN 'Introversion' -- I => Introversion
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),1,1)) = 'E' THEN 'Extraversion' -- E => Extraversion
          ELSE NULL                                              -- Otherwise NULL
        END AS mbti_1,                                           -- Alias mbti_1
        CASE                                                     -- MBTI letter 2 -> Sens/Intuit
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL -- Guard
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),2,1)) = 'S' THEN 'Sensing'  -- S => Sensing
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),2,1)) = 'N' THEN 'Intuition'-- N => Intuition
          ELSE NULL                                              -- Otherwise NULL
        END AS mbti_2,                                           -- Alias mbti_2
        CASE                                                     -- MBTI letter 3 -> Think/Feel
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL -- Guard
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),3,1)) = 'T' THEN 'Thinking' -- T => Thinking
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),3,1)) = 'F' THEN 'Feeling'  -- F => Feeling
          ELSE NULL                                              -- Otherwise NULL
        END AS mbti_3,                                           -- Alias mbti_3
        CASE                                                     -- MBTI letter 4 -> Judge/Perceive
          WHEN ewp.mbti IS NULL OR length(trim(ewp.mbti)) < 4 THEN NULL -- Guard
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),4,1)) = 'J' THEN 'Judging'  -- J => Judging
          WHEN UPPER(SUBSTRING(TRIM(ewp.mbti),4,1)) = 'P' THEN 'Perceiving' -- P => Perceiving
          ELSE NULL                                              -- Otherwise NULL
        END AS mbti_4                                            -- Alias mbti_4
      FROM emp_with_names ewp                                    -- Source rows for parsing
    ),                                                           -- End CTE parsed_splits
    final_raw AS (                                               -- CTE: assemble raw output
      SELECT                                                     -- Select final fields (pre-dedup)
        ewp.employee_id,                                         -- Employee key
        ewp.full_name,                                           -- Full name
        ewp.directorate,  -- directorate name                    -- Directorate name (already resolved)
        ewp.role,         -- role/position name                  -- Role name (already resolved)
        ewp.grade,        -- grade name                          -- Grade name (already resolved)
        COALESCE(cl.score, 0)  AS score,                         -- Latest competency; default 0
        COALESCE(ewp.pauli, 0) AS pauli,                         -- Pauli; default 0
        ps.disc_1,                                                -- DISC part 1
        ps.disc_2,                                                -- DISC part 2
        ps.mbti_1,                                                -- MBTI dim 1
        ps.mbti_2,                                                -- MBTI dim 2
        ps.mbti_3,                                                -- MBTI dim 3
        ps.mbti_4,                                                -- MBTI dim 4
        COALESCE(ewp.iq,  0)   AS iq,                            -- IQ; default 0
        COALESCE(ewp.gtq, 0)   AS gtq,                           -- GTQ; default 0
        COALESCE(ewp.tiki,0)   AS tiki,                          -- TIKI; default 0
        ewp.strengths          AS strengths,                     -- Top strength theme
        ewp.scale_code,                                          -- PAPI scale code
        COALESCE(ewp.papi_scores, 0) AS papi_scores              -- PAPI score; default 0
      FROM emp_with_names ewp                                    -- Source: names + PAPI attached
      LEFT JOIN competencies_latest cl USING (employee_id)       -- Add latest competency by employee
      LEFT JOIN parsed_splits     ps USING (employee_id)         -- Add parsed DISC/MBTI fields
    )                                                            -- End CTE final_raw
    SELECT DISTINCT                                              -- Final projection with deduplication
      employee_id,                                               -- Employee key
      full_name,                                                 -- Full name
      directorate,                                               -- Directorate
      role,                                                      -- Role
      grade,                                                     -- Grade
      score,                                                     -- Competency score
      pauli,                                                     -- Pauli score
      disc_1,                                                    -- DISC part 1
      disc_2,                                                    -- DISC part 2
      mbti_1,                                                    -- MBTI dim 1
      mbti_2,                                                    -- MBTI dim 2
      mbti_3,                                                    -- MBTI dim 3
      mbti_4,                                                    -- MBTI dim 4
      iq,                                                        -- IQ
      gtq,                                                       -- GTQ
      tiki,                                                      -- TIKI
      strengths,                                                 -- Top strength theme
      scale_code,                                                -- PAPI scale code
      papi_scores                                                -- PAPI score
    FROM final_raw                                               -- Read from assembled CTE
    ORDER BY employee_id, full_name;                             -- Stable, readable ordering by employee id and name
    """
    
    # Open a secure PostgreSQL connection using environment variables
    conn = psycopg2.connect(
        host=os.getenv("host"),        # Database host
        port=os.getenv("port"),        # Database port (e.g., 5432)
        dbname=os.getenv("database"),  # Database name
        user=os.getenv("user"),        # Database user
        password=os.getenv("password"),# Database password
        sslmode="require",             # Enforce SSL/TLS encryption
        connect_timeout=30,            # Fail fast if unreachable
    )
    try:
        # Create a cursor for executing SQL
        with conn.cursor() as cur:     # Ensure cleanup via context manager
            cur.execute(qry)           # Run the query (no parameters for this version)
            rows = cur.fetchall()      # Fetch all result rows
            cols = [d[0] for d in cur.description]  # Extract column names
        return pd.DataFrame(rows, columns=cols)     # Build and return the DataFrame
    finally:
        conn.close()                   # Always close the DB connection


In [38]:
# Call function get_all_employee_data
df_all_employees = get_all_employee_data(schema="public")
df_all_employees

,employee_id,full_name,directorate,role,grade,score,pauli,disc_1,disc_2,mbti_1,mbti_2,mbti_3,mbti_4,iq,gtq,tiki,strengths,scale_code,papi_scores
0,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,4.0,55,Conscientious,Dominant,Introversion,Sensing,Feeling,Perceiving,0,0,8,Learner,Papi_A,6.0
1,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,4.0,55,Conscientious,Dominant,Introversion,Sensing,Feeling,Perceiving,0,0,8,Learner,Papi_B,0
2,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,4.0,55,Conscientious,Dominant,Introversion,Sensing,Feeling,Perceiving,0,0,8,Learner,Papi_C,1.0
3,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,4.0,55,Conscientious,Dominant,Introversion,Sensing,Feeling,Perceiving,0,0,8,Learner,Papi_D,4.0
4,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,4.0,55,Conscientious,Dominant,Introversion,Sensing,Feeling,Perceiving,0,0,8,Learner,Papi_E,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40195,EMP101999,Adit Mahendra,Technology,Data Analyst,III,4.0,76,Dominant,Conscientious,Introversion,Intuition,Feeling,Perceiving,90.0,25.0,8,Ideation,Papi_T,8.0
40196,EMP101999,Adit Mahendra,Technology,Data Analyst,III,4.0,76,Dominant,Conscientious,Introversion,Intuition,Feeling,Perceiving,90.0,25.0,8,Ideation,Papi_V,4.0
40197,EMP101999,Adit Mahendra,Technology,Data Analyst,III,4.0,76,Dominant,Conscientious,Introversion,Intuition,Feeling,Perceiving,90.0,25.0,8,Ideation,Papi_W,9.0
40198,EMP101999,Adit Mahendra,Technology,Data Analyst,III,4.0,76,Dominant,Conscientious,Introversion,Intuition,Feeling,Perceiving,90.0,25.0,8,Ideation,Papi_X,8.0


## Develop Python Code

### Calculate TV-TGV Match Rate

In [39]:
# PAPI letters used (K & Z are handled with special formulas)                     # context note for readers
PAPI_LETTERS = ['A','B','C','D','E','F','G','I','K','L','N','O','P','R','S','T','V','W','X','Z']  # fixed set of PAPI scales

# Create function to calculate TV TGV
def calculate_tv_tgv(df_employee_benchmarks: pd.DataFrame,
                     df_all_employees: pd.DataFrame,
                     round_digits: int = 2) -> pd.DataFrame:
    """
    Returns a DataFrame containing, per employee:
      employee_id, full_name, directorate, role, grade,
      tv_pauli, tv_iq, tv_gtq, tv_tiki, tv_disc, tv_mbti, tv_strengths,
      tv_papi_kostick,
      tgv_motivation_drive, tgv_leadership_influence, tgv_creativity_innovation,
      tgv_social_orientation, tgv_adaptability_stress, tgv_conscient_reliable,
      tgv_cognitive, tgv_culture_values
    """ 

    # ---------------------------
    # Helper
    # ---------------------------
    def _mode_nonnull(s: pd.Series):
        vc = s.dropna().value_counts()                                           # count non-null occurrences
        return vc.index[0] if not vc.empty else None                             # return most frequent value or None

    def _safe_pct(user_val, bench_val):
        if pd.isna(bench_val) or bench_val == 0:                                 # guard against division by zero/NaN
            return np.nan                                                        # return NaN when no benchmark
        return (user_val / bench_val) * 100.0                                    # percentage vs benchmark (0–100+)

    def _papi_letter(x):
        if pd.isna(x):                                                           # handle missing scale_code
            return None
        s = str(x).upper().strip()                                               # normalize to uppercase string
        m = re.search(r'([A-Z])$', s)                                            # try last letter (e.g., "...A")
        if m and m.group(1) in PAPI_LETTERS:                                     # if last letter is valid PAPI
            return m.group(1)
        m = re.search(r'PAPI[_\-\s]?([A-Z])', s)                                 # try pattern like "PAPI_A" or "PAPI A"
        if m and m.group(1) in PAPI_LETTERS:                                     # validate matched letter
            return m.group(1)
        return None                                                              # unknown pattern -> None

    def _disc_has(df: pd.DataFrame, style: str) -> pd.Series:
        return ((df['disc_1'].astype(str) == style) |                            # true if disc_1 equals style
                (df['disc_2'].astype(str) == style)).astype(int) * 100.0        # or disc_2 equals style -> 0/100

    def _strength_is(df: pd.DataFrame, theme: str) -> pd.Series:
        return (df['strengths'].astype(str) == theme).astype(int) * 100.0        # 1 if theme matches else 0, scaled to 0/100

    # ---------------------------
    # Benchmark single-row per employee (avoid duplicates from long PAPI)
    # ---------------------------
    bench_emp = df_employee_benchmarks.drop_duplicates(subset=['employee_id']).copy()  # unique employees for stats

    # Numeric medians                                                                   # compute central tendency
    med_pauli = pd.to_numeric(bench_emp['pauli'], errors='coerce').median()            # median Pauli
    med_iq    = pd.to_numeric(bench_emp['iq'],    errors='coerce').median()            # median IQ
    med_gtq   = pd.to_numeric(bench_emp['gtq'],   errors='coerce').median()            # median GTQ
    med_tiki  = pd.to_numeric(bench_emp['tiki'],  errors='coerce').median()            # median TIKI

    # Categorical modes (reference for tv_disc / tv_mbti / tv_strengths)               # most frequent benchmark labels
    bench_disc1 = _mode_nonnull(bench_emp['disc_1'])                                   # mode DISC part 1
    bench_disc2 = _mode_nonnull(bench_emp['disc_2'])                                   # mode DISC part 2
    bench_mbti1 = _mode_nonnull(bench_emp['mbti_1'])                                   # mode MBTI dim 1
    bench_mbti2 = _mode_nonnull(bench_emp['mbti_2'])                                   # mode MBTI dim 2
    bench_mbti3 = _mode_nonnull(bench_emp['mbti_3'])                                   # mode MBTI dim 3
    bench_mbti4 = _mode_nonnull(bench_emp['mbti_4'])                                   # mode MBTI dim 4
    bench_strengths = _mode_nonnull(bench_emp['strengths'])                            # mode Strength theme

    # PAPI medians per letter (use long form)                                          # benchmark per PAPI letter
    bench_papi = df_employee_benchmarks[['scale_code','papi_scores']].copy()           # pick needed columns
    bench_papi['letter'] = bench_papi['scale_code'].map(_papi_letter)                  # parse letter from scale_code
    bench_papi['papi_scores'] = pd.to_numeric(bench_papi['papi_scores'], errors='coerce')  # numeric scores
    papi_medians = (bench_papi.dropna(subset=['letter'])                                # drop rows w/o letter
                              .groupby('letter')['papi_scores']                         # group by PAPI letter
                              .median())                                                # median per letter

    # ---------------------------
    # Base data: one row per employee from all_employees
    # ---------------------------
    base_cols = ['employee_id','full_name','directorate','role','grade',              # identity columns
                 'pauli','iq','gtq','tiki','disc_1','disc_2',                         # raw numeric + DISC parts
                 'mbti_1','mbti_2','mbti_3','mbti_4','strengths']                     # MBTI dims + strengths
    base = (df_all_employees[base_cols]                                               # select required columns
            .drop_duplicates(subset=['employee_id'])                                   # ensure unique per employee
            .copy())                                                                   # work on a copy

    for c in ['pauli','iq','gtq','tiki']:                                             # iterate numeric fields
        base[c] = pd.to_numeric(base[c], errors='coerce')                              # coerce to numeric

    # --- TV numeric ------------------------------------------------------------------
    base['tv_pauli'] = base['pauli'].apply(lambda x: _safe_pct(x, med_pauli)).clip(upper=100)  # cap at 100
    base['tv_iq']    = base['iq'].apply(   lambda x: _safe_pct(x, med_iq)).clip(upper=100)     # IQ vs median
    base['tv_gtq']   = base['gtq'].apply(  lambda x: _safe_pct(x, med_gtq)).clip(upper=100)    # GTQ vs median
    base['tv_tiki']  = base['tiki'].apply( lambda x: _safe_pct(x, med_tiki)).clip(upper=100)   # TIKI vs median

    # --- TV DISC (0/50/100) ---
    def _tv_disc_row(row):
        match = int(str(row['disc_1']) == str(bench_disc1)) + int(str(row['disc_2']) == str(bench_disc2))      # +1 if disc_1 or disc_2 matches mode
        return (match / 2.0) * 100.0
    base['tv_disc'] = base.apply(_tv_disc_row, axis=1)                                                         # compute per row

    # --- TV MBTI (0/25/50/75/100) ----------------------------------------------------
    def _tv_mbti_row(row):
        m = 0                                                                          # match counter
        m += int(str(row['mbti_1']) == str(bench_mbti1))                               # check dim 1
        m += int(str(row['mbti_2']) == str(bench_mbti2))                               # check dim 2
        m += int(str(row['mbti_3']) == str(bench_mbti3))                               # check dim 3
        m += int(str(row['mbti_4']) == str(bench_mbti4))                               # check dim 4
        return (m / 4.0) * 100.0                                                       # scale to percentage

    base['tv_mbti'] = base.apply(_tv_mbti_row, axis=1)                                 # compute per row

    # --- TV strengths (0/100) vs benchmark mode --------------------------------------
    base['tv_strengths'] = (base['strengths'].astype(str) == str(bench_strengths)).astype(int) * 100.0  # 0/100

    # ---------------------------
    # PAPI user (long -> wide), then compute tv_papi_*
    # ---------------------------
    all_papi = df_all_employees[['employee_id','scale_code','papi_scores']].copy()     # user PAPI rows
    all_papi['letter'] = all_papi['scale_code'].map(_papi_letter)                      # parse letter
    all_papi['papi_scores'] = pd.to_numeric(all_papi['papi_scores'], errors='coerce')  # ensure numeric

    user_papi_wide = (all_papi.dropna(subset=['letter'])                               # keep valid letters
                      .pivot_table(index='employee_id', columns='letter',               # pivot to wide format
                                   values='papi_scores', aggfunc='mean'))              # mean if duplicates

    tv_papi = pd.DataFrame(index=user_papi_wide.index)                                 # init output frame
    for L in PAPI_LETTERS:                                                             # iterate each PAPI letter
        col = f'tv_papi_{L}'                                                           # output column name
        if L in user_papi_wide.columns and L in papi_medians.index:                    # only if both user & bench available
            bench = papi_medians[L]                                                    # benchmark median
            if L in ['K','Z']:                                                         # special inverse formula for K/Z
                tv_papi[col] = user_papi_wide[L].apply(
                    lambda x: np.nan if pd.isna(bench) or bench == 0                   # guard against invalid bench
                    else ((2*bench - x)/bench)*100.0                                   # inverted ratio * 100
                )
            else:
                tv_papi[col] = user_papi_wide[L].apply(lambda x: _safe_pct(x, bench))  # standard ratio * 100
        else:
            tv_papi[col] = np.nan                                                      # missing either side -> NaN
    tv_papi = tv_papi.reset_index()                                                    # bring employee_id back as column

    # Merge tv_papi back to base                                                         
    out = base.merge(tv_papi, on='employee_id', how='left')                            # left join to keep all employees

    # Kostick = average of all tv_papi_*                                                # overall PAPI fit score
    papi_cols = [c for c in out.columns if c.startswith('tv_papi_')]                   # collect generated columns
    out['tv_papi_kostick'] = out[papi_cols].mean(axis=1, skipna=True)                  # row-wise mean ignoring NaN

    # ---------------------------
    # TGV components (indicator 0/100 + numeric TVs)
    # ---------------------------
    # DISC indicators                                                                     # 0/100 presence flags
    disc_dom  = _disc_has(out, 'Dominant')                                              # has Dominant
    disc_inf  = _disc_has(out, 'Influencer')                                            # has Influencer
    disc_ste  = _disc_has(out, 'Steadiness')                                            # has Steadiness
    disc_con  = _disc_has(out, 'Conscientious')                                         # has Conscientious

    # MBTI indicators                                                                     # example alignment rules
    # Interpretation: match benchmark on EI axis (mbti_1). To force "Extroversion",       # explain customization
    # replace this with (out['mbti_1'].eq('Extroversion').astype(int)*100.0)              # alternative logic note
    mbti1_ei  = (out['mbti_1'].astype(str) == str(bench_mbti1)).astype(int) * 100.0     # 0/100 EI match vs mode
    mbti2_int = (out['mbti_2'].astype(str).str.lower() == 'intuition').astype(int) * 100.0  # 0/100 prefers Intuition

    # Strength indicators                                                                  # binary flags by theme
    s_Achiever        = _strength_is(out, 'Achiever')                                    # Achiever 0/100
    s_Arranger        = _strength_is(out, 'Arranger')                                    # Arranger 0/100
    s_Command         = _strength_is(out, 'Command')                                     # Command 0/100
    s_SelfAssurance   = _strength_is(out, 'Self-Assurance')                              # Self-Assurance 0/100
    s_Developer       = _strength_is(out, 'Developer')                                   # Developer 0/100
    s_Futuristic      = _strength_is(out, 'Futuristic')                                  # Futuristic 0/100
    s_Ideation        = _strength_is(out, 'Ideation')                                    # Ideation 0/100
    s_Communication   = _strength_is(out, 'Communication')                               # Communication 0/100
    s_Woo             = _strength_is(out, 'Woo')                                         # Woo 0/100
    s_Relator         = _strength_is(out, 'Relator')                                     # Relator 0/100
    s_Adaptability    = _strength_is(out, 'Adaptability')                                # Adaptability 0/100
    s_Deliberative    = _strength_is(out, 'Deliberative')                                # Deliberative 0/100
    s_Discipline      = _strength_is(out, 'Discipline')                                  # Discipline 0/100
    s_Connectedness   = _strength_is(out, 'Connectedness')                               # Connectedness 0/100
    s_Analytical      = _strength_is(out, 'Analytical')                                  # Analytical 0/100
    s_StrengthsTheme  = _strength_is(out, 'Strengths')                                   # literal 'Strengths' theme, per spec
    s_Belief          = _strength_is(out, 'Belief')                                      # Belief 0/100

    # Helper to average components (ignore NaN)                                           # utility for TGV components
    def _avg(cols):
        return pd.concat(cols, axis=1).mean(axis=1, skipna=True)                         # row-wise mean

    out['tgv_motivation_drive'] = _avg([out['tv_pauli'], out['tv_papi_A'], s_Achiever])  # Motivation & Drive
    out['tgv_leadership_influence'] = _avg([                                             # Leadership & Influence
        mbti1_ei, disc_dom, out['tv_papi_L'], out['tv_papi_P'],
        s_Arranger, s_Command, s_SelfAssurance, s_Developer
    ])
    out['tgv_creativity_innovation'] = _avg([mbti2_int, out['tv_papi_Z'], s_Futuristic, s_Ideation])  # Creativity/Innovation
    out['tgv_social_orientation']    = _avg([disc_inf, out['tv_papi_S'], s_Communication, s_Woo, s_Relator])  # Social Orientation
    out['tgv_adaptability_stress']   = _avg([disc_ste, out['tv_papi_T'], out['tv_papi_E'], s_Adaptability])   # Adaptability/Stress
    out['tgv_conscient_reliable']    = _avg([disc_con, out['tv_papi_C'], out['tv_papi_D'], s_Deliberative, s_Discipline])  # Conscientious/Reliable
    out['tgv_cognitive']             = _avg([out['tv_iq'], out['tv_gtq'], out['tv_tiki'],                                      # Cognitive composite
                                             out['tv_papi_I'], s_Connectedness, s_Analytical, s_StrengthsTheme])
    out['tgv_culture_values']        = s_Belief                                           # Culture & Values (single component)

    # ---------------------------
    # Finishing: rounding & column order
    # ---------------------------
    wanted = [                                                                         # final column order
        'employee_id','full_name','directorate','role','grade',
        'tv_pauli','tv_iq','tv_gtq','tv_tiki','tv_disc','tv_mbti','tv_strengths','tv_papi_kostick',
        'tgv_motivation_drive','tgv_leadership_influence','tgv_creativity_innovation',
        'tgv_social_orientation','tgv_adaptability_stress','tgv_conscient_reliable',
        'tgv_cognitive','tgv_culture_values'
    ]

    num_cols = [c for c in wanted if c not in ['employee_id','full_name','directorate','role','grade']]  # numeric fields
    out[wanted] = out[wanted]                                                        # ensure all wanted columns exist (no-op if yes)
    out[num_cols] = out[num_cols].round(round_digits)                                # round numeric outputs

    return out[wanted]                                                               # return final, ordered DataFrame

In [40]:
# Call function calculate_tv_tgv
tv_tgv = calculate_tv_tgv(df_employee_benchmarks, df_all_employees)
tv_tgv

,employee_id,full_name,directorate,role,grade,tv_pauli,tv_iq,tv_gtq,tv_tiki,tv_disc,...,tv_strengths,tv_papi_kostick,tgv_motivation_drive,tgv_leadership_influence,tgv_creativity_innovation,tgv_social_orientation,tgv_adaptability_stress,tgv_conscient_reliable,tgv_cognitive,tgv_culture_values
0,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,87.30,0.00,0.00,100.0,100.0,...,0.0,113.49,109.10,42.86,50.00,36.0,102.50,45.78,16.48,0.0
1,DUP2096,Putri Wibowo,HR & Corp Affairs,HRBP,III,42.86,0.00,0.00,50.0,50.0,...,0.0,104.77,27.62,42.86,38.89,32.0,61.67,85.33,20.33,0.0
2,DUP2722,Dewi Syahputra,HR & Corp Affairs,Sales Supervisor,IV,100.00,100.00,67.92,25.0,0.0,...,0.0,115.29,60.00,23.21,33.33,24.0,74.17,65.78,47.34,0.0
3,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,100.00,82.08,83.02,75.0,100.0,...,100.0,109.05,100.00,64.29,33.33,12.0,37.50,61.78,54.08,0.0
4,DUP3452,Fitri Anugrah Subekti,Technology,Finance Officer,V,95.24,84.91,100.00,87.5,50.0,...,0.0,104.99,125.08,58.93,27.78,56.0,29.17,48.00,43.31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,EMP101995,Utami Kusuma Hidayat,Commercial,Sales Supervisor,IV,41.27,100.00,100.00,100.0,0.0,...,0.0,119.30,93.76,23.21,47.22,4.0,91.67,72.44,62.64,100.0
2006,EMP101996,Adit Maulana,HR & Corp Affairs,Data Analyst,V,82.54,0.00,90.57,75.0,0.0,...,0.0,138.89,134.18,37.50,63.89,28.0,55.83,40.00,39.04,0.0
2007,EMP101997,Satya Halim Putra,Technology,Brand Executive,IV,36.51,0.00,0.00,12.5,0.0,...,0.0,95.30,132.17,25.00,44.44,20.0,0.00,78.67,17.17,0.0
2008,EMP101998,Kurnia Subekti,Technology,Sales Supervisor,III,66.67,0.00,71.70,25.0,50.0,...,0.0,100.75,122.22,30.36,50.00,16.0,43.33,28.00,16.01,0.0


### Formatting Outputs

In [41]:
# Create function to format TV & TGV Output
def formatting_tv_tgv(tv_tgv: pd.DataFrame, round_digits: int = 2,
                      w_tv: float = 0.60, w_tgv: float = 0.40) -> pd.DataFrame:
    """
    Build a per-employee summary of TV & TGV.
    final_match_rate = w_tv * tv_match_rate + w_tgv * tgv_match_rate (defaults 0.60 / 0.40).
    If one component is NaN, final_match_rate equals the available component.
    """  

    tv_map = {                                                     # label map for TV components (display names)
        'tv_pauli': 'Pauli',
        'tv_mbti': 'MBTI',
        'tv_disc': 'DISC',
        'tv_iq': 'IQ',
        'tv_gtq': 'GTQ',
        'tv_tiki': 'TIKI',
        'tv_strengths': 'CliftonStrengths',
        'tv_papi_kostick': 'PAPI Kostick',
    }
    tgv_map = {                                                    # label map for TGV components (display names)
        'tgv_motivation_drive': 'Motivation & Drive',
        'tgv_leadership_influence': 'Leadership & Influence',
        'tgv_creativity_innovation': 'Creativity & Innovation Orientation',
        'tgv_social_orientation': 'Social Orientation & Collaboration',
        'tgv_adaptability_stress': 'Adaptability & Stress Tolerance',
        'tgv_conscient_reliable': 'Conscientiousness & Reliability',
        'tgv_cognitive': 'Cognitive Complexity & Problem-Solving',
        'tgv_culture_values': 'Cultural & Values Urgency',
    }

    tv_cols  = list(tv_map.keys())                                 # ordered TV column names to aggregate
    tgv_cols = list(tgv_map.keys())                                # ordered TGV column names to aggregate

    # Ensure all required columns exist; if missing, create as NaN so means won’t error
    for c in tv_cols + tgv_cols:                                   # iterate expected columns
        if c not in tv_tgv.columns:                                # if not present in input frame
            tv_tgv[c] = np.nan                                     # create with NaN values

    def non_zero_tgv_names(row):                                   # collect non-zero/non-NaN TGV labels for each row
        names = []                                                 # start list of labels
        for c in tgv_cols:                                         # loop over TGV columns
            val = row.get(c)                                       # fetch value safely
            if pd.notna(val) and float(val) != 0.0:               # include only if numeric and non-zero
                names.append(tgv_map[c])                           # map key -> human-readable label
        return names                                               # return list (may be empty)

    tv_match  = tv_tgv[tv_cols].mean(axis=1, skipna=True)          # row-wise average across TV columns (NaN ignored; all-NaN -> NaN)
    tgv_match = tv_tgv[tgv_cols].mean(axis=1, skipna=True)         # row-wise average across TGV columns (NaN ignored; all-NaN -> NaN)

    out = pd.DataFrame({                                           # assemble output frame
        'employee_id'   : tv_tgv['employee_id'],                   # identity: employee id
        'full_name'     : tv_tgv['full_name'],                     # identity: name
        'directorate'   : tv_tgv['directorate'],                   # identity: directorate
        'role'          : tv_tgv['role'],                          # identity: role
        'grade'         : tv_tgv['grade'],                         # identity: grade
        'tgv_name'      : tv_tgv.apply(non_zero_tgv_names, axis=1),# list of present TGV component names per row
        'tv_name'       : [list(tv_map.values())] * len(tv_tgv),   # same TV label list repeated for each row
        'tv_match_rate' : tv_match.round(round_digits),            # rounded TV average
        'tgv_match_rate': tgv_match.round(round_digits),           # rounded TGV average
    })

    # Weighted final score (60% TV, 40% TGV by default); if either side is NaN, use the other
    def weighted_final(tv_val, tgv_val):                           # combine two components with fallbacks
        if pd.isna(tv_val) and pd.isna(tgv_val):                   # both missing
            return np.nan                                          # result missing
        if pd.isna(tv_val):                                        # only TV missing
            return tgv_val                                         # use TGV
        if pd.isna(tgv_val):                                       # only TGV missing
            return tv_val                                          # use TV
        return w_tv * tv_val + w_tgv * tgv_val                     # both present -> weighted sum

    out['final_match_rate'] = [                                    # compute for each row
        weighted_final(tv, tg) for tv, tg in zip(out['tv_match_rate'], out['tgv_match_rate'])
    ]
    out['final_match_rate'] = out['final_match_rate'].round(round_digits)  # round final score

    return out                                                     # return formatted summary

In [42]:
# Call function formatting_tv_tgv
final_results = formatting_tv_tgv(tv_tgv, round_digits=2)
final_results 

,employee_id,full_name,directorate,role,grade,tgv_name,tv_name,tv_match_rate,tgv_match_rate,final_match_rate
0,DUP1942,Dewi Fadillah Hidayat,Commercial,Brand Executive,IV,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",59.47,50.34,55.82
1,DUP2096,Putri Wibowo,HR & Corp Affairs,HRBP,III,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",37.20,38.59,37.76
2,DUP2722,Dewi Syahputra,HR & Corp Affairs,Sales Supervisor,IV,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",54.15,40.98,48.88
3,DUP3015,Bayu Kurniawan,Technology,Brand Executive,III,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",93.64,45.37,74.33
4,DUP3452,Fitri Anugrah Subekti,Technology,Finance Officer,V,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",74.70,48.53,64.23
...,...,...,...,...,...,...,...,...,...,...
2005,EMP101995,Utami Kusuma Hidayat,Commercial,Sales Supervisor,IV,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",63.82,61.87,63.04
2006,EMP101996,Adit Maulana,HR & Corp Affairs,Data Analyst,V,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",51.50,49.80,50.82
2007,EMP101997,Satya Halim Putra,Technology,Brand Executive,IV,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",24.29,39.68,30.45
2008,EMP101998,Kurnia Subekti,Technology,Sales Supervisor,III,"[Motivation & Drive, Leadership & Influence, C...","[Pauli, MBTI, DISC, IQ, GTQ, TIKI, CliftonStre...",39.26,38.24,38.85


### Export Output

In [43]:
# Export final output to Excel
final_results.to_excel('Export_Output_Step2.xlsx', index=False)

## Solution Justification

* **I didn’t cram everything into one huge SQL.** Putting all steps in a single query would make it very heavy and slow. It’s hard to read, hard to fix, and won’t scale when the data grows.

* **I also avoided one giant chain of SQL blocks.** Long step-after-step SQL makes the database do a lot of extra work. I keep SQL small and focused.

* **I still use small SQL blocks (CTEs) where needed.** I use them only to fetch two clean datasets: all employees and the benchmark employees. That keeps things tidy and traceable.

* **I move the tricky math to Python on purpose.** The rules mix numbers (medians), categories (exact match vs. no match), special cases (PAPI K/Z), and text cleanup (different “PAPI_A” spellings). Python handles this clearly and safely; doing it all in SQL would be messy and risky.

* **This split runs faster and costs less.** The database does what it’s good at to getting the data. Python does the custom scoring. That avoids heavy, slow SQL that wastes compute.

* **It’s easier to maintain.** Adding a new TV/TGV, changing a weight, or fixing a rule is a few readable lines in Python instead of editing a long, fragile SQL script.

* **It’s easier to test.** I can check each rule (DISC 0/50/100, MBTI steps, K/Z formula) with small unit tests. That’s much harder with one giant SQL.

* **I don’t show `baseline_score` and `user_score` in the final table.** We already use them to compute the match. Showing them for every TV/TGV would explode the number of rows and make the result hard to use. If needed, we can expose a separate “audit” view later.

* **The final table fits what managers need.** One row per employee with TV and TGV match rates, plus the overall match. It’s simple to read and easy to plug into a dashboard.

* **Weights are supported.** We can keep the default (e.g., final = 60% TV + 40% TGV) or use custom weights without touching SQL.

* **It scales well.** As we add more people, benchmarks, or TVs, the database work stays light, and Python can handle the extra scoring smoothly.

* **Given the tight deadline, I chose a practical solution.** A clean split between SQL (data in) and Python (logic) delivers results fast without chasing perfection.
